In [1]:
# | default_exp core.execution_loop

In [2]:
# | export
from typing import Dict, Any, List, Optional, Callable
from dataclasses import dataclass
from enum import Enum
import time
import json


class LoopState(Enum):
    IDLE = "idle"
    RUNNING = "running"
    PAUSED = "paused"
    STOPPED = "stopped"
    ERROR = "error"


@dataclass
class ExecutionStep:
    """Single step in execution loop"""
    step_id: str
    agent_name: str
    message: str
    result: Optional[Dict[str, Any]] = None
    error: Optional[str] = None
    timestamp: Optional[str] = None
    duration: Optional[float] = None


@dataclass
class LoopConfig:
    """Configuration for execution loop"""
    max_iterations: int = 10
    timeout_seconds: int = 300
    auto_continue: bool = False
    require_approval: bool = True
    error_handling: str = "stop"  # stop, continue, retry


class ExecutionLoop:
    """Autonomous execution loop for agents"""
    
    def __init__(self, config: LoopConfig):
        self.config = config
        self.state = LoopState.IDLE
        self.steps: List[ExecutionStep] = []
        self.current_iteration = 0
        self.start_time: Optional[float] = None
        self.approval_callback: Optional[Callable] = None
        self.progress_callback: Optional[Callable] = None
    
    def set_approval_callback(self, callback: Callable[[ExecutionStep], bool]):
        """Set callback for step approval"""
        self.approval_callback = callback
    
    def set_progress_callback(self, callback: Callable[[ExecutionStep], None]):
        """Set callback for progress updates"""
        self.progress_callback = callback
    
    def run(self, agent, initial_message: str, **kwargs) -> Dict[str, Any]:
        """Run the execution loop"""
        self.state = LoopState.RUNNING
        self.start_time = time.time()
        self.current_iteration = 0
        
        current_message = initial_message
        
        try:
            while self._should_continue():
                step = self._execute_step(agent, current_message, **kwargs)
                
                if step.error and self.config.error_handling == "stop":
                    self.state = LoopState.ERROR
                    break
                
                # Check if agent wants to continue
                if not self._agent_wants_to_continue(step.result):
                    break
                
                # Prepare next message
                current_message = self._prepare_next_message(step.result)
                
                self.current_iteration += 1
            
            self.state = LoopState.STOPPED
            
        except Exception as e:
            self.state = LoopState.ERROR
            self._add_error_step(str(e))
        
        return self._get_summary()
    
    def _execute_step(self, agent, message: str, **kwargs) -> ExecutionStep:
        """Execute a single step"""
        step_id = f"step_{self.current_iteration + 1}"
        step = ExecutionStep(
            step_id=step_id,
            agent_name=agent.config.name,
            message=message,
            timestamp=self._get_timestamp()
        )
        
        # Request approval if required
        if self.config.require_approval and self.approval_callback:
            if not self.approval_callback(step):
                step.error = "Step cancelled by user"
                self.steps.append(step)
                return step
        
        start_time = time.time()
        
        try:
            # Execute agent
            result = agent.run(message, **kwargs)
            step.result = result
            step.duration = time.time() - start_time
            
        except Exception as e:
            step.error = str(e)
            step.duration = time.time() - start_time
        
        self.steps.append(step)
        
        # Notify progress
        if self.progress_callback:
            self.progress_callback(step)
        
        return step
    
    def _should_continue(self) -> bool:
        """Check if loop should continue"""
        if self.state != LoopState.RUNNING:
            return False
        
        if self.current_iteration >= self.config.max_iterations:
            return False
        
        if self.start_time and (time.time() - self.start_time) > self.config.timeout_seconds:
            return False
        
        return True
    
    def _agent_wants_to_continue(self, result: Optional[Dict[str, Any]]) -> bool:
        """Check if agent indicates it wants to continue"""
        if not result:
            return False
        
        # Look for continuation signals in the result
        content = result.get("content", "").lower()
        
        # Stop signals
        stop_signals = ["task complete", "finished", "done", "no further action"]
        if any(signal in content for signal in stop_signals):
            return False
        
        # Continue signals
        continue_signals = ["next step", "continuing", "proceeding", "tool_calls"]
        if any(signal in content for signal in continue_signals):
            return True
        
        # If there are tool calls, likely wants to continue
        if result.get("tool_calls"):
            return True
        
        return False
    
    def _prepare_next_message(self, result: Optional[Dict[str, Any]]) -> str:
        """Prepare message for next iteration"""
        if not result:
            return "Continue with the next step."
        
        # Extract key information from result
        content = result.get("content", "")
        tool_results = []
        
        if result.get("tool_calls"):
            for tool_call in result["tool_calls"]:
                if tool_call.get("result"):
                    tool_results.append(f"Tool {tool_call['function']['name']}: {tool_call['result']}")
        
        next_message = "Based on the previous step:\n"
        if content:
            next_message += f"Response: {content}\n"
        if tool_results:
            next_message += f"Tool Results: {'; '.join(tool_results)}\n"
        next_message += "Continue with the next appropriate step."
        
        return next_message
    
    def _add_error_step(self, error: str):
        """Add error step to history"""
        step = ExecutionStep(
            step_id=f"error_{len(self.steps) + 1}",
            agent_name="system",
            message="Error occurred",
            error=error,
            timestamp=self._get_timestamp()
        )
        self.steps.append(step)
    
    def _get_summary(self) -> Dict[str, Any]:
        """Get execution summary"""
        total_duration = time.time() - self.start_time if self.start_time else 0
        
        return {
            "state": self.state.value,
            "total_steps": len(self.steps),
            "successful_steps": len([s for s in self.steps if not s.error]),
            "failed_steps": len([s for s in self.steps if s.error]),
            "total_duration": total_duration,
            "iterations": self.current_iteration,
            "steps": [self._step_to_dict(step) for step in self.steps]
        }
    
    def _step_to_dict(self, step: ExecutionStep) -> Dict[str, Any]:
        """Convert step to dictionary"""
        return {
            "step_id": step.step_id,
            "agent_name": step.agent_name,
            "message": step.message,
            "result": step.result,
            "error": step.error,
            "timestamp": step.timestamp,
            "duration": step.duration
        }
    
    def _get_timestamp(self) -> str:
        """Get current timestamp"""
        from datetime import datetime
        return datetime.now().isoformat()
    
    def pause(self):
        """Pause execution"""
        self.state = LoopState.PAUSED
    
    def resume(self):
        """Resume execution"""
        if self.state == LoopState.PAUSED:
            self.state = LoopState.RUNNING
    
    def stop(self):
        """Stop execution"""
        self.state = LoopState.STOPPED


class AutonomousAgent:
    """Agent wrapper with autonomous execution capabilities"""
    
    def __init__(self, agent, loop_config: Optional[LoopConfig] = None):
        self.agent = agent
        self.loop_config = loop_config or LoopConfig()
        self.execution_loop = ExecutionLoop(self.loop_config)
    
    def run_autonomous(self, task: str, **kwargs) -> Dict[str, Any]:
        """Run agent autonomously on a task"""
        return self.execution_loop.run(self.agent, task, **kwargs)
    
    def set_approval_callback(self, callback: Callable):
        """Set approval callback for autonomous execution"""
        self.execution_loop.set_approval_callback(callback)
    
    def set_progress_callback(self, callback: Callable):
        """Set progress callback for autonomous execution"""
        self.execution_loop.set_progress_callback(callback)
